<a href="https://colab.research.google.com/github/Usool-Data-Science/Organization-Analysis/blob/main/Multilabel%20text%20classification%20with%20tensorflow%20and%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import io
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import files
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
uploaded = files.upload()
phrases = pd.read_excel(io.BytesIO(uploaded['Test of AI ML possibilities.xlsx']),skiprows = 108,names = ['phrase','class']) #we dropped the numeric
#complete = phrases['']
phrases.head()

Saving Test of AI ML possibilities.xlsx to Test of AI ML possibilities.xlsx


,phrase,class
0,- asset management,4
1,- center,4
2,- co,4
3,- developer,4
4,- end development,4


In [ ]:
phrases['phrase'].value_counts()

## Data Exploration and splitting

In [46]:
labelled_phrases = phrases[~phrases['class'].isna()]
unlabelled_phrases = phrases[phrases['class'].isna()]
#print(f'Total phrases: {len(phrases)} \nTotal labelled phrases: {len(labelled_phrases)} \nTotal unlabelled phrases: {len(unlabelled_phrases)} \
#      \n%Labelled: {len(labelled_phrases)/len(phrases)*100}')

print('Total phrases: {0} \nTotal labelled phrases: {1} \nTotal unlabelled phrases: {2} \nPercentage Labelled: %{3:.2f}'.format(len(phrases),len(labelled_phrases),len(unlabelled_phrases),len(labelled_phrases)/len(phrases)*100))


Total phrases: 41523 
Total labelled phrases: 7626 
Total unlabelled phrases: 33897 
Percentage Labelled: %18.37


In [6]:
# Defining some constants/hyperparameters
BUFFER_SIZE = 1000 # for reshuffling
BATCH_SIZE = 100
NUM_EPOCHS = 20

In [47]:
#aa = labelled_phrases[labelled_phrases['class']=='4'].loc[:]['class'] = 4
index_to_change = labelled_phrases[labelled_phrases['class']=='e'].index
labelled_phrases.at[index_to_change,'class'] = 4

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [ ]:
labelled_phrases['phrase'].value_counts()

In [ ]:
labelled_phrases['class'].value_counts()

In [48]:
import random
labelled_phrases = labelled_phrases.sample(frac = 1, replace = False, random_state = 12345)
sentences = labelled_phrases['phrase']
labels = labelled_phrases['class']

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

### manual shuffling

x_train  = sentences[0:training_size]
#x_train = random.sample(sentences, k = training_size)
x_test = sentences[training_size:]
y_train  = labels[0:training_size]
y_test = labels[training_size:]
x_future = unlabelled_phrases['phrase']

In [49]:
y_train.value_counts()

3    2960
4    2153
2     503
1     484
Name: class, dtype: int64

In [50]:
vocab_size = 1000
embedding_dim = 16
max_words = 10000
num_classes = max(y_train) + 1

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)
future_sequence = tokenizer.texts_to_sequences(x_future)

x_train = tokenizer.sequences_to_matrix(train_sequences, mode='binary')
x_test = tokenizer.sequences_to_matrix(test_sequences, mode='binary')
x_future = tokenizer.sequences_to_matrix(future_sequence, mode='binary')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train[0])
print(len(x_train[0]))

print(y_train[0])
print(len(y_train[0]))

[0. 0. 0. ... 0. 0. 0.]
10000
[0. 0. 0. 1. 0.]
5


In [59]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

#print(decode_review(padded[1]))
#print(training_sentences[1])

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Defining early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0, 
    restore_best_weights = True
    )
print(model.metrics_names)

[]


In [53]:
batch_size = 20
epochs = 20

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, callbacks = [early_stopping], verbose=1, validation_split=0.1)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
275/275 [==============================] - 2s 6ms/step - loss: 0.8920 - accuracy: 0.6217 - val_loss: 0.6618 - val_accuracy: 0.6869
Epoch 2/20
275/275 [==============================] - 1s 4ms/step - loss: 0.5447 - accuracy: 0.7566 - val_loss: 0.6604 - val_accuracy: 0.6754
Epoch 3/20
275/275 [==============================] - 1s 4ms/step - loss: 0.3329 - accuracy: 0.9168 - val_loss: 0.7351 - val_accuracy: 0.6689
Epoch 4/20
275/275 [==============================] - 1s 4ms/step - loss: 0.1993 - accuracy: 0.9297 - val_loss: 0.8373 - val_accuracy: 0.5311
77/77 - 0s - loss: 0.6334 - accuracy: 0.6959 - 181ms/epoch - 2ms/step
Test loss: 0.6334216594696045
Test loss: 0.00
Test accuracy: 0.6959370970726013
Test loss: 0.00


In [ ]:
prob = model.predict(x_future)
future_class = []
for i in range(len(prob)):
  future_class.append(np.argmax(prob[i]))
#future_class

In [55]:
newphrase = unlabelled_phrases['phrase']
predicted_data = pd.DataFrame({'phrase': newphrase, 'class':future_class})
predicted_data

,phrase,class
45,.net 3.5,4
46,.net 4.0,4
47,.net 5,4
48,.net 6,4
50,.net architecture,4
...,...,...
41518,zwave,3
41519,zxing,3
41520,zynq,3
41521,zynq soc,3


## NETWORK VISUALIZATION

In [56]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 512)


In [60]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [62]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
len(phrases)

41523

In [70]:
final_phrases = pd.merge(phrases.dropna(), predicted_data, how = 'outer') 
final_phrases

,phrase,class
0,- asset management,4
1,- center,4
2,- co,4
3,- developer,4
4,- end development,4
...,...,...
41518,zwave,3
41519,zxing,3
41520,zynq,3
41521,zynq soc,3


In [72]:
final_phrases.to_csv('complete_data.csv', index = False, encoding='utf-8')
files.download('complete_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>